In [1]:
# Constants

GPS_DATA_PATH = "gps_data.csv"

In [2]:
# Clean the GPS data exported from the phone app

import pandas as pd


# Read the CSV file
df = pd.read_csv(GPS_DATA_PATH, parse_dates=["loggingTime(txt)"])

# Discard rows without GPS data
df_gps = df[
    [
        "loggingSample(N)",
        "locationTimestamp_since1970(s)",
        "locationLatitude(WGS84)",
        "locationLongitude(WGS84)",
    ]
]
df_gps = df_gps.dropna(subset=["locationLatitude(WGS84)", "locationLongitude(WGS84)"])
df_gps = df_gps.reset_index(drop=True)

first_sample_time = df_gps["locationTimestamp_since1970(s)"].iloc[0]
df_gps["timeSinceStart(s)"] = df_gps["locationTimestamp_since1970(s)"] - first_sample_time

In [3]:
# Visualize the GPS data on a map

import folium


# Create a map centered on the mean latitude and longitude
center_lat = df_gps["locationLatitude(WGS84)"].mean()
center_lon = df_gps["locationLongitude(WGS84)"].mean()
m = folium.Map(
    location=[center_lat, center_lon],
    zoom_start=21,
    max_zoom=22,
    tiles="https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
    attr="Google",
    name="Google Satellite",
)

folium.TileLayer("OpenStreetMap").add_to(m)
folium.LayerControl().add_to(m)

# Add markers for each GPS point
for idx, row in df_gps.iterrows():
    folium.Marker(
        location=[row["locationLatitude(WGS84)"], row["locationLongitude(WGS84)"]],
        popup=f"Sample: {row['loggingSample(N)']}",
        tooltip=f"Time delta: {row['timeSinceStart(s)']}",
    ).add_to(m)

# Add a PolyLine to connect the points
locations = df_gps[["locationLatitude(WGS84)", "locationLongitude(WGS84)"]].values.tolist()
folium.PolyLine(locations, weight=2, color="red").add_to(m)

# Save the map
# m.save("gps_data_map.html")

# Display the map
m